In [3]:
import numpy as np                                                                        
import matplotlib.pyplot as plt
import pyCloudy as pc
import pyneb as pn
from astropy    import constants as const
from astropy.io import ascii
import pandas as pd
from scipy import interpolate
import warnings
from scipy.integrate import quad, IntegrationWarning
# from scipy.integrate import simps
from numpy import log10, exp

kpc = const.kpc.cgs.value

atom = ['Lya','HeII','CIV']

def read_file(path_way, atom):
    Mod = pc.CloudyModel(path_way, cloudy_version_major=23)
    Mod.ionic_names

    N_H = sum(Mod.dr*Mod.nH)
    # solar_metallicitiy from von Steiger et al. 2016
    frac_He = 1.0E-01
    frac_C = 4.47E-04
    frac_O = 6.61E-04
    frac_N = 9.33E-05
    frac_Mg = 7.08E-05

    N_HI = sum(Mod.dr*Mod.nH*Mod.get_ionic('H',0))
    N_HII = sum(Mod.dr*Mod.nH*Mod.get_ionic('H',1))
    N_HeII = frac_He*sum(Mod.dr*Mod.nH*Mod.get_ionic('He',1))
    N_OVI = frac_O*sum(Mod.dr*Mod.nH*Mod.get_ionic('O',5))
    N_NV = frac_N*sum(Mod.dr*Mod.nH*Mod.get_ionic('N',4))
    N_CIV = frac_C*sum(Mod.dr*Mod.nH*Mod.get_ionic('C',3))

    num = len(Mod.nH)
    r_CIV = path_way +  '.ele_C'
    f = open(r_CIV,'r')
    header = f.readline()
    CIV_frac = np.zeros(num)
    i = 0
    for line in f:
        line = line.strip()
        columns = line.split()
        j = float(columns[4])
        CIV_frac[i] = j
        i = i + 1

    r_He = path_way +  '.ele_He'
    f = open(r_He,'r')
    header = f.readline()
    HeII_frac = np.zeros(num)
    i = 0
    for line in f:
        line = line.strip()
        columns = line.split()
        j = float(columns[2])
        HeII_frac[i] = j
        i = i + 1



    n_H= Mod.nH
    n_He = n_H*frac_He
    n_C = n_H*frac_C
    nden_CIV = CIV_frac*n_C
    nden_HeII = HeII_frac*n_He

    # tt = nden_CIV / n_H
    # print(tt.mean())

    if atom == 'CIV':
        Cloudy_Lum = float(Mod.get_emis_vol('C__4_154819A')) + float(Mod.get_emis_vol('C__4_155078A'))
        Cloudy_emis = (Mod.get_emis('C__4_154819A')) + (Mod.get_emis('C__4_155078A'))
        Cloudy_den = nden_CIV
    elif atom == 'Lya':
        Cloudy_Lum= float(Mod.get_emis_vol('H__1_121567A'))
        Cloudy_emis = Mod.get_emis('H__1_121567A')
        Cloudy_den = n_H
    elif atom == 'HeII':
        Cloudy_Lum = float(Mod.get_emis_vol('HE_2_164043A'))
        Cloudy_emis = Mod.get_emis('HE_2_164043A')
        Cloudy_den = nden_HeII
    return Cloudy_Lum , Cloudy_emis ,Cloudy_den

def radius(path, atom):
    Mod = pc.CloudyModel(path, cloudy_version_major=23)
    radius = Mod.radius/kpc
    radius_kpc =Mod.radius 
    dr = Mod.dr 
    return radius, radius_kpc, dr

def make_data_file(path,atom):
    lum ,emis ,den = read_file(path,atom)
    radius_R , radius_kpc , dr=  radius(path,atom)
    tt =  pd.DataFrame(np.column_stack((radius_R,emis,den)))
    tt.to_csv('/home/jin/T_Cloudy/{}_cloudy.txt'.format(atom), sep='\t',index=False,header =False)
    # tt.to_csv('/home/jin/RT_code/CIV_cloudy.txt', sep='\t',index = False,header=False)
    return print("make data file!")

def SB(z,radius_kpc, emissivity):
    r_min, r_max = radius_kpc.min(), radius_kpc.max()       
    Project_R = np.linspace(0, r_max, 100)
    surface_brightness = np.zeros(len(Project_R))
    Lumin = np.zeros(len(Project_R))
    T_Lumin = np.zeros(len(Project_R))
    
    for ii, bb in enumerate(Project_R[:-1]):
        array_R = np.where(radius_kpc >= bb)[0]
        r_surface_brightness = 0
        for dR in (array_R[:-1]):
            dr = radius_kpc[dR+1] - radius_kpc[dR]
            r_surface_brightness += emissivity[dR] * dr
        surface_brightness[ii] = 4*r_surface_brightness / (1+z)**4
        dR_pro = Project_R[ii+1] -Project_R[ii] 
        Lumin[ii] = 2*np.pi*bb*surface_brightness[ii] * dR_pro * (1+z)**4

    return Project_R, surface_brightness, Lumin

def RT_SB(path):
    name = ['radius','SB_K','SB_H','SB_tot','1','2','3']
    data_sp = pd.read_csv(path, sep='\s+', header=None,names=name)
    rad, SB_t, SB_k,SB_h =  data_sp['radius'].to_numpy(),data_sp['SB_tot'].to_numpy(),data_sp['SB_K'].to_numpy(),data_sp['SB_H'].to_numpy()
    return rad/kpc, rad, SB_t 

def find_y(x_find,x,y):
    ii = int(np.where(x <= x_find)[0][-1])
    # print(ii)
    y_find = (y[ii+1] -y[ii]) / (x[ii+1] - x[ii])*(x_find - x[ii]) + y[ii]
    return y_find

warnings.filterwarnings("ignore", category=IntegrationWarning)

In [ ]:
# path_LT = r'/home/jin/T_Cloudy/OTS_n_LT_y/CIV_Lumin_42'
# make_data_file(path_LT ,'Lya')

make data file!


In [8]:
#Ubuntu

path_LT = r'/home/jin/T_Cloudy/OTS_n_LT_y/CIV_Lumin_42'

v_rand = ['11.8','50']
v_out =['0','300','600']
v_emit = ['50','100']

def RT_make_parameter(v_out,v_emit ,v_ran,atom):
    rand = int(v_rand * 10)
    if v_out == 0 :
        expand = "000"
        vout_order = 0
    else: 
        expand = v_out 
        vout_order = 2

        if v_emit < 100 :
            emit = int(v_emit*10)
            emit_order = 1
        else : 
            emit = v_emit
            emit_order = 2
    
            globals()['path_rt_{}_{}_{}'.format(v_out,v_out,v_emit)] =  r'/home/jin/T_Cloudy/sb_cloudy/data_RT/N_atom000E+00_Vexp{}E+0{}_Vemit{}E+0{}_tauD000E+00_Vran{}E+01radi.dat'.format(expand, vout_order, emit,emit_order,rand)
            globals()['rt_radius_{}_{}_{}'.format(v_out,v_out,v_emit)] , globals()['rt_radius_kpc_{}_{}_{}'.format(v_out,v_out,v_emit)], globals()['rt_sb_t_{}_{}_{}'.format(v_out,v_out,v_emit)] = RT_SB(globals()['path_rt_{}_{}_{}'].format(v_out,v_out,v_emit))
    return print("make parameters of v_rand = {} km/s,  v_exp = {} km/s , and v_emit = {} km/s  ".format(v_out,v_out,v_emit))

def CLOUDY_make_parameter(atom):
    z_red_Shift = 0
    globals()['radius_{}'.format(atom)] , globals()['radius_kpc_{}'.format(atom)], globals()['dr_{}'.format(atom)] = radius(path_LT,atom)
    globals()['{}_lum'.format(atom)] , globals()['{}_emis'.format(atom)] , globals()['{}_den'.format(atom)] = read_file(path_LT,atom)
    globals()['p_radius_{}'.format(atom)] , globals()['sb_{}'.format(atom)], globals()['lum_sb_{}'.format(atom)] = SB(z_red_Shift,globals()['radius_kpc_{}'.format(atom)],globals()['{}_emis'.format(atom)] )
    return print("make parameters of {} ".format(atom))

In [10]:
# path_radi =  '/home/jin/RT_code/data_CIV/N_atom000E+00_Vexp000E+00_Vemit100E+00_tauD000E+00_Vran118E+01radi.dat'
# name = ['radius','SB_K','SB_H','SB_tot','1','2','3']
# data_sp = pd.read_csv(path_radi, sep='\s+', header=None,names=name)

# rad, SB_t, SB_k,SB_h =  data_sp['radius'].to_numpy(),data_sp['SB_tot'].to_numpy(),data_sp['SB_K'].to_numpy(),data_sp['SB_H'].to_numpy()

# path_sc =  '/home/jin/RT_code/data_CIV/N_atom100E+01_Vexp000E+00_Vemit100E+00_tauD000E+00_Vran118E+01radi.dat'
# data_sc = pd.read_csv(path_sc, sep='\s+', header=None,names=name)

# rad_sc, SB_t_sc, SB_k_sc,SB_h_sc =  data_sc['radius'].to_numpy(),data_sc['SB_tot'].to_numpy(),data_sc['SB_K'].to_numpy(),data_sc['SB_H'].to_numpy()


# z = 1 # z = 14 로 설정해야 R = 1kpc 에서 photoionization 이랑 RT-scat 결과의 SB가 같게 나옴.
# SB_z_civ =SB_civ / (1+z)**3 / SI_sr_to_arc
# SB_z_heii =SB_heii / (1+z)**3 / SI_sr_to_arc
# SB_z_lya =SB_lya / (1+z)**3 / SI_sr_to_arc

# fig = plt.figure(1,figsize=(21,7))
# plt.subplot(131)
# plt.plot(Rad_civ,SB_z_civ,'b-',label=r'CIV Photoionization at z = {:}'.format(z))
# plt.plot(Rad_heii,SB_z_heii,'r-',label='HeII')
# plt.plot(rad/kpc,SB_t,'bs',mfc='None',alpha=0.5,label='RT-Scat w/o scattering')
# plt.plot(rad_sc/kpc,SB_t_sc,'bo',mfc='None',alpha=1,label='RT-Scat w scattering')

# plt.axvline(x=1,color='k',linestyle='--')
# plt.xlim(0.4,5)
# plt.ylim(1e-26,1e-11)
# plt.yscale('log')
# plt.xlabel('Radius [kpc]',fontsize=25)
# plt.ylabel(r'SB $\rm [erg/s/cm^{2}/arcsec^{2}]$',fontsize=25)
# plt.xticks(fontsize=20)
# plt.yticks(fontsize=20)
# plt.minorticks_on()
# plt.grid(True)
# plt.legend(fontsize=15)


# plt.subplot(132)
# plt.plot(Rad_civ,SB_z_civ,'b-',label=r'CIV Photoionization at z = {:}'.format(z))
# plt.plot(Rad_heii,SB_z_heii,'r-',label='HeII')
# plt.plot(rad/kpc,SB_t,'bs',mfc='None',alpha=0.5,label='RT-Scat w/o scattering')
# plt.plot(rad_sc/kpc,SB_t_sc,'bo',mfc='None',alpha=1,label='RT-Scat w scattering')
# plt.axvline(x=7, color='g',linestyle='--')
# plt.axvline(x=1,color='k',linestyle='--')
# plt.xlim(0.4,25)
# plt.ylim(1e-26,1e-11)
# plt.yscale('log')
# plt.xlabel('Radius [kpc]',fontsize=25)
# plt.ylabel(r'SB $\rm [erg/s/cm^{2}/arcsec^{2}]$',fontsize=25)
# plt.xticks(fontsize=20)
# plt.yticks(fontsize=20)
# plt.minorticks_on()
# plt.grid(True)
# plt.legend(fontsize=15)



# plt.subplot(133)
# plt.plot(Rad_civ,SB_z_civ,'b-',label=r'CIV Photoionization at z = {:}'.format(z))
# plt.plot(Rad_heii,SB_z_heii,'r-',label='HeII')
# plt.plot(rad/kpc,SB_t,'bs',mfc='None',alpha=0.5,label='RT-Scat w/o scattering')
# plt.plot(rad_sc/kpc,SB_t_sc,'bo',mfc='None',alpha=1,label='RT-Scat w scattering')
# plt.axvline(x=7, color='g',linestyle='--')
# plt.axvline(x=1,color='k',linestyle='--')
# plt.xlim(0.4,100)
# plt.ylim(1e-26,1e-11)
# plt.yscale('log')
# plt.xlabel('Radius [kpc]',fontsize=25)
# plt.ylabel(r'SB $\rm [erg/s/cm^{2}/arcsec^{2}]$',fontsize=25)
# plt.xticks(fontsize=20)
# plt.yticks(fontsize=20)
# plt.minorticks_on()
# plt.grid(True)
# plt.legend(fontsize=15)

# plt.tight_layout()
# plt.savefig('/home/jin/바탕화면/CIV_RT_Scat_Result.pdf',dpi=100)